# Analyse av metadata som finnes i JSON data

In [ ]:
import pathlib

from rich.prompt import Prompt

index_file = pathlib.Path("index.json")
if not index_file.exists():
    raw_path = Prompt.ask("Velg index fil")
    index_file = pathlib.Path(raw_path)

In [ ]:
import polars as pl

df = pl.read_json(index_file)
df.describe()

In [ ]:
df.group_by("type").agg(pl.len()).plot.bar("type", "len")

In [ ]:
df.group_by("language").agg(pl.len()).plot.bar("language", "len")